In [1]:
import re
import pandas as pd

In [2]:
content = ''
with open('./data/本草綱目.txt', 'r', encoding='utf8') as file:
    for line in file:
        content += line
finds = re.findall(r'    \n(?:(^\S.*)(?:\n    \n))?(^\S.*)\n((?:    .*\n?)+)(?=    \n?)', content, re.M)
df = pd.DataFrame(columns=['目錄', '標題', '內文'])

for find in finds:
    df_row = dict((key,'') for key in df.columns)
    df_row['目錄'] = '\n'.join([t.strip() for t in find[0].strip().split('\n')])
    df_row['標題'] = '\n'.join([t.strip() for t in find[1].strip().split('\n')])
    df_row['內文'] = '\n'.join([t.strip() for t in find[2].strip().split('\n')])
    df = df.append(df_row, ignore_index=True)

In [3]:
df[df.iloc[:,0]==''].index

Int64Index([   0,    1,    2,    4,    5,    6,    7,    8,    9,   10,   11,
              12,   13,   14,   15,   16,   17,   18,   19,   20,   21,  191,
            1717, 1729, 1873],
           dtype='int64')

In [5]:
df.head()

,目錄,標題,內文
0,,《本草綱目》原序,紀稱∶望龍光，知古劍；觇寶氣，辨明珠。故萍實商羊，非天明莫洞。厥後博物稱華，辨字稱康，析寶玉...
1,,進《本草綱目》疏,湖廣黃州府儒學增廣生員李建元謹奏，爲遵奉明例訪書，進獻《本草》以備採擇事。臣伏讀禮部儀制司勘...
2,,《本草綱目》凡例,一、《神農本草》三卷，三百六十種，分上、中、下三品。梁陶弘景增藥一倍，随品附入。唐、宋重脩，...
3,序例上,歷代諸家本草,《神農本草經》 掌禹錫曰∶舊說《本草經》三卷，神農所作，而不經見，《漢書·藝文誌》亦無錄焉。...
4,,引據古今醫家書目,時珍曰∶自陶弘景以下，唐、宋諸本草引用醫書，凡八十四家，而唐慎微居多。時珍今所引，除舊本外，...


In [4]:
df.to_csv('./data/本草綱目.csv', encoding='utf-8-sig')

In [6]:
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        row['內文'] = re.sub(r'【 【主治】', '【主治】', row['內文'])
        row['內文'] = re.sub(r'主治 【附方', '主治】 【附方', row['內文'])
        row['內文'] = re.sub(r'附方 吐血', '附方】 吐血', row['內文'])
        row['內文'] = re.sub(r'【别名】', '【釋名】', row['內文'])
        row['內文'] = re.sub(r'【主冶】', '【主治】', row['內文'])
df.to_csv('./data/本草綱目_Ver1.csv', encoding='utf-8-sig')

In [8]:
string = '''塵芒入目∶大藕洗搗，綿裹，滴汁入目中，即出也。（《普濟方》）
藕   【釋名】藕絲菜（五、六月嫩時，採爲蔬【氣味】甘，平，無毒。
【主治】生食，主霍亂後虛渴煩悶不能食，解酒食毒（蘇頌）。
功與藕同（時珍）。解煩毒，下瘀血（汪穎）。
藕節  【氣味】澀，平，無毒。
大明曰∶冷。伏硫黃。
'''
re.sub(r'(?:(\w+ ?\w*)  (【(?:.*)))', r'\\x\1\\x\n', string)

'塵芒入目∶大藕洗搗，綿裹，滴汁入目中，即出也。（《普濟方》）\n\\x藕 \\x\n\n【主治】生食，主霍亂後虛渴煩悶不能食，解酒食毒（蘇頌）。\n功與藕同（時珍）。解煩毒，下瘀血（汪穎）。\n\\x藕節\\x\n\n大明曰∶冷。伏硫黃。\n'

In [9]:
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if not re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        continue
    if re.search(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M):
        ns = re.sub(r'(?:(^\w{1,10})(?: {2,2}【.*))', r'\\x\1\\x\n', row['內文'])
        print(ns)
#     sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))|(?:(^\w{2,10})(?: {1,1}【.*))', row['內文'], re.M)
#     sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M)

（《别錄》中品）
【校正】并入《開寶》生銀。
【釋名】白金（《綱目》）、鋈。
時珍曰∶《爾雅》∶白金謂之銀，其美者曰 。《說文》雲∶鋈，白金也。梵書謂之阿路巴。
【集解】《别錄》曰∶銀屑生永昌，採無時。
弘景曰∶銀之所出處，亦與金同，但是生土中也。煉餌法亦似金。永昌屬益州，今屬寧州。
恭曰∶銀與金，生不同處，所在皆有，而以虢州者爲勝，此外多鉛穢爲劣。高麗作帖者，雲非銀礦所出，然色青不如虢州者。
誌曰∶生銀出饒州樂平諸坑銀礦中，狀如硬錫，紋理粗錯自然者真。
頌曰∶銀在礦中與銅相雜，土人採得，以鉛再三煎煉方成，故爲熟銀。生銀則生銀礦中，狀如硬錫。其金坑中所得，乃在土石中滲漏成條，若絲發狀，土人謂之老翁須，極難得。
方書用生銀，必得此乃真。
曰∶按《南越誌》∶波斯國有天生藥銀，用爲試藥指環。又燒朱粉甕下，多年沉積有銀，號杯鉛銀，光軟甚好，與波斯銀功力相似，隻是難得。今時燒煉家，每一斤生鉛，隻得一、二铢。《山海經》雲∶東北樂平郡堂少山出銀甚多。黔中生銀體硬，不堪入藥。
宗 曰∶銀出于礦，須煎煉成，故名熟銀。其生銀即不自礦中出而特然生者，又謂之老翁須，其入用不同。世之朮士，以朱砂而成，以鉛汞而成，以焦銅而成者，既無造化之氣，豈可入藥，不可不别。
時珍曰∶閩、浙、荊、湖、饒、信、廣、滇、貴州諸處，山中皆産銀，有礦中煉出者，有沙土中煉出者。其生銀，俗稱銀筍、銀牙者也，亦曰出山銀。獨孤滔《丹房鏡源》所謂鉛坑中出褐色石，形如筍，打破即白，名曰自然牙，曰自然鉛，亦曰生鉛，此有變化之道，不堪服食者，是也。《管子》雲∶上有鉛，下有銀。《地鏡圖》雲∶山有蔥，下有銀。銀之氣，入夜正白，流散在地，其精變爲白雄雞。《寶藏論》雲∶銀有十七種，又外有四種。天生牙，生銀坑内石縫中，狀如亂絲，色紅者上，入火紫白如草根者次之，銜黑石者最奇，生樂平、鄱陽産鉛之山，一名龍牙，一名龍須，是正生銀，無毒，爲至藥根本也。生銀生石礦中，成片塊、大小不定，狀如硬錫。母砂銀，生五溪丹砂穴中，色理紅光。黑鉛銀，得子母之氣。
此四種爲真銀。有水銀銀、草砂銀、曾青銀、石綠銀、雄黃銀、雌黃銀、硫黃銀、膽礬銀、靈草銀，皆是以藥制成者；丹陽銀、銅銀、鐵銀、白錫銀，皆以藥點化者，十三種皆假銀也。
外有四種∶新羅銀、波斯銀、林邑銀、雲南銀，并精好。
銀屑  【脩治】弘景曰∶醫方鎮心丸用之，不可正服。爲屑，當以水銀研，令消也。
恭曰∶方家用

（《拾遺》）
【釋名】鼹鼠（音偃）、偃鼠【集解】弘景注鼹鼠曰∶諸山林中，有獸大如水牛，形似豬，灰赤色，下腳似象，胸前尾上皆白，有力而鈍，亦名隱鼠。人取食之，肉亦似牛，多以作脯。乃雲是鼠王，其精溺一滴落地，輒成一鼠，災年則多出也。
藏器曰∶此是獸類，非鼠之俦。大如牛而前腳短，皮入 轡用。《莊子》所謂鼹鼠飲河，不過滿腹者。陶言是鼠王，精滴成鼠。遍訪山人無其說，亦不能土中行。此乃妄說，陶誤信爾。頌曰∶鼹鼠出滄州及胡中。似牛而鼠首黑足，大者千斤。多伏于水，又能堰水放沫。彼人食其肉。
時珍曰∶按《異物誌》雲∶鼠母頭腳似鼠，口銳毛蒼，大如水牛而畏狗。見則主水災。《晉書》雲∶宣城郡出隱鼠，大如牛，形似鼠，腳類象而驢蹄。毛灰赤色，胸前尾上白色。有力而鈍。《金樓子》雲∶晉寧縣境出大鼠，狀如牛，土人謂之偃鼠。時出山遊，毛落田間，悉成小鼠，苗稼盡耗。《梁書》雲∶倭國有山鼠如牛，又有大蛇能吞之。據此則隱鼠非無，而陶說有本；諸家闢之太甚者，未深考耳。又《爾雅》雲∶ 身似鼠而馬蹄，長須而賊，一歲千斤，秦人謂之“小驢”者，即此物也。
膏  【主治】痔 惡瘡（陶弘景）。
（音石。《綱目》）
【釋名】碩鼠（與 同。出《周易》）、 鼠（音酌。出《廣雅》）、雀鼠（出《埤雅》）、鼠（音俊。出《唐韻》）。
時珍曰∶碩，大也，似鼠而大也。關西方音轉 爲 ，訛 爲雀。蜀人謂之 鼠，取其毛作筆。俊亦大也。
【集解】時珍曰∶ 鼠處處有之，居土穴、樹孔中。
形大于鼠，頭似兔，尾有毛，青黃色。善鳴，能人立，交前兩足而舞。好食粟、豆，與鼢鼠俱爲田害。鼢小居田，而 大居山也。範成大雲∶賓州 鼠專食山豆根，土人取其腹乾之入藥，名 鼠肚。陸玑謂此亦有五技，與蝼蛄同名者，誤矣。
肚  【氣味】甘，寒，無毒。
【主治】咽喉痹痛，一切熱氣，研末含咽，神效（時珍。出《虞衡
（《拾遺》）
【釋名】 （音駝撥）、答刺不花（出《正要》）。
時珍曰∶按《唐書》有 鼠，即此也。 ，言其肥也。《唐韻》作 ，音仆朴，俗訛爲土撥耳。蒙古人名答刺不花。
【集解】藏器曰∶土撥鼠，生西番山澤間，穴土爲窠。形如獺。夷人掘取食之。《魏略》雲∶“大秦國出闢毒鼠”，近似此也。
肉  【氣味】甘，平，無毒。
時珍曰∶按《飲膳正要》雲∶雖肥而煮之無油，味短，多食難克化，微動風。
【主治】野雞 瘡，煮食肥美宜人（藏器）。
頭骨  【主治】小兒夜卧不寧，懸之枕邊，即安（時

In [10]:
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if not re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        continue
#     if re.match(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M):
#         ns = re.sub(r'(?:(^\w{1,10})(?: {2,2}【.*))', r'\\x\1\\x\n', row['內文'], re.M)
#         print(ns)
#     sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))|(?:(^\w{2,10})(?: {1,1}【.*))', row['內文'], re.M)
    sites = re.findall(r'(?:(^\w{1,10})(?: {2,2}【.*))', row['內文'], re.M)
#     if sites:
#         print(sites)


In [23]:
new_df = pd.DataFrame(columns=['目錄', '標題', '釋名', '內文', '部位', '部位內文', '主治', '氣味', '附方'])
for index, row in df.iterrows():
    if pd.isnull(row['目錄']):
        continue
    if not re.search(r'\w+[部|目]第\w+卷 \w{1,2}之\w+', row['目錄']):
        continue
    data_row = dict((k, '') for k in new_df.columns)
    data_row['目錄'] = row['目錄']
    data_row['標題'] = row['標題']
    data_row['內文'] = row['內文']
    if re.search(r'【釋名】(.*?)(?=【|(?<=.$))', data_row['內文'], re.S):
        data_row['釋名'] = re.search(r'【釋名】(.*?)(?=【|(?<=.$))', data_row['內文'], re.S).group(1)
    newContent = re.sub(r'(?:(\w{1,10})  (【(?:.*)))', r'\\x\1\\x\n\2', data_row['內文'], re.M)
    sites = re.findall(r'(\\x.+?\\x.*?)(?=\\x|(?<=.$))', newContent, re.S)
    sites = sites if sites else [data_row['內文']]
    for s_content in sites:
        data_row['部位內文'] = s_content
        if re.search(r'\\x(.+?)\\x', s_content, re.S):
            data_row['部位'] = re.search(r'\\x(.+?)\\x', s_content, re.S).group(1)
        if re.search(r'【主治】(.*?)(?=【|(?<=.$))', s_content, re.S):
            data_row['主治'] = re.search(r'【主治】(.*?)(?=【|(?<=.$))', s_content, re.S).group(1)
        if re.search(r'【氣味】(.*?)(?=【|(?<=.$))', s_content, re.S):
            data_row['氣味'] = re.search(r'【氣味】(.*?)(?=【|(?<=.$))', s_content, re.S).group(1)
        if re.search(r'【附方】(.*?)(?=【|(?<=.$))', s_content, re.S):
            data_row['附方'] = re.search(r'【附方】(.*?)(?=【|(?<=.$))', s_content, re.S).group(1)
        new_df = new_df.append(data_row, ignore_index=True)


In [28]:
new_df.loc[np.where(new_df.loc[:,'標題'] == '燒酒')]

,目錄,標題,釋名,內文,部位,部位內文,主治,氣味,附方
886,谷部第二十五卷 谷之四,燒酒,火酒（《綱目》）、阿剌吉酒（《,（《綱目》）\n【釋名】火酒（《綱目》）、阿剌吉酒（《【集解】 時珍曰∶燒酒非古法也。自元時...,,（《綱目》）\n【釋名】火酒（《綱目》）、阿剌吉酒（《【集解】 時珍曰∶燒酒非古法也。自元時...,消冷積寒氣，燥濕痰，開郁結，止水洩，治霍亂瘧疾噎膈，心腹死，殺蟲闢瘴，利小便，堅大便，洗赤目...,辛、甘，大熱，有大毒。\n時珍曰∶過飲敗胃傷膽，喪心損夀，甚則黑腸腐胃而死。與薑、蒜同食，令...,新七。\n冷氣心痛∶燒酒入飛鹽飲，即止。\n陰毒腹痛∶燒酒溫飲，汗出即止。\n嘔逆不止∶真火...


In [22]:
new_df

,目錄,標題,釋名,內文,部位,部位內文,主治,氣味,附方
0,水部第五卷 水之一,燒酒,（時珍曰）∶地氣升爲雲，天氣降爲雨，故人之汗，以天地之雨名之。\n,（《拾遺》）\n【釋名】（時珍曰）∶地氣升爲雲，天氣降爲雨，故人之汗，以天地之雨名之。\n【...,,（《拾遺》）\n【釋名】（時珍曰）∶地氣升爲雲，天氣降爲雨，故人之汗，以天地之雨名之。\n【...,夫妻各飲一杯，還房，當穫時有子，神效。（藏器）宜煎發散及補中 益氣藥。（時珍）,鹹，平，無毒。\n立春雨水,
1,水部第五卷 水之一,燒酒,時珍曰∶降注雨水謂之潦，又淫雨爲潦。韓退之詩雲，潢潦無根源，朝灌夕已除，是矣。\n,（《綱目》）\n【釋名】時珍曰∶降注雨水謂之潦，又淫雨爲潦。韓退之詩雲，潢潦無根源，朝灌夕已...,,（《綱目》）\n【釋名】時珍曰∶降注雨水謂之潦，又淫雨爲潦。韓退之詩雲，潢潦無根源，朝灌夕已...,煎調脾胃、去濕熱之藥（時珍）。\n,甘，平，無毒。\n,
2,水部第五卷 水之一,燒酒,時珍曰∶露者，陰氣之液也，夜氣着物而潤澤于道旁也。\n,（《拾遺》）\n【釋名】時珍曰∶露者，陰氣之液也，夜氣着物而潤澤于道旁也。\n【氣味】甘，平...,,（《拾遺》）\n【釋名】時珍曰∶露者，陰氣之液也，夜氣着物而潤澤于道旁也。\n【氣味】甘，平...,秋露繁時，以盤收取，煎如饴，令人延年不饑（藏器）。禀肅殺之氣，宜煎潤肺 殺祟之藥，及調疥癬蟲...,甘，平，無毒。\n,
3,水部第五卷 水之一,燒酒,膏露（《綱目》）、瑞露（《綱目》）、天酒（《綱目》）、神漿。時珍曰∶按《瑞應圖》雲∶甘露，美...,（《拾遺》）\n【釋名】膏露（《綱目》）、瑞露（《綱目》）、天酒（《綱目》）、神漿。時珍曰∶...,,（《拾遺》）\n【釋名】膏露（《綱目》）、瑞露（《綱目》）、天酒（《綱目》）、神漿。時珍曰∶...,食之潤五臟，長年，不饑，神仙（藏器）。,甘，大寒，無毒。\n,
4,水部第五卷 水之一,燒酒,,（《拾遺》）\n【集解】藏器曰∶生巴西絕域中，狀如饧也。時珍曰∶按《方國誌》雲∶大食國秋時收...,,（《拾遺》）\n【集解】藏器曰∶生巴西絕域中，狀如饧也。時珍曰∶按《方國誌》雲∶大食國秋時收...,胸膈諸熱，明目止渴。（藏器）,甘，平，無毒。\n,
5,水部第五卷 水之一,燒酒,方諸水。藏器曰∶方諸，大蚌也。熟摩令熱，向月取之，得水二三合，亦如朝露。陽燧向日，方諸向月，...,（《拾遺》）\n【釋名】方諸水。藏器曰∶方諸，大蚌也。熟摩令熱，向月取之，得水二三合，亦如朝...,,（《拾遺》）\n【釋名】方諸水。藏器曰∶方諸，大蚌也。熟摩令熱，向月取之，得水二三合，亦如朝...,明目定心，去小兒煩熱，止渴（藏器）。,甘，寒，無毒。\n,
6,水部第五卷 水之一,燒酒,時珍曰∶陰盛則露凝爲霜，霜能殺物而露能滋物，性随時異也。乾象占雲∶天氣下降而爲露，清風薄之而...,（《拾遺》）\n【釋名】時珍曰∶陰盛則露凝爲霜，霜能殺物而露能滋物，性随時異也。乾象占雲∶天...,,（《拾遺》）\n【釋名】時珍曰∶陰盛則露凝爲霜，霜能殺物而露能滋物，性随時異也。乾象占雲∶天...,食之解酒熱，傷寒鼻塞，酒後諸熱面赤者（藏器）。和蚌粉，敷暑月痱瘡，及腋 下赤腫，立瘥（陳承）。\n,甘，寒，無毒。\n,新一。寒熱瘧疾（秋後霜一錢半，熱酒服之。《集玄方》）
7,水部第五卷 水之一,燒酒,時珍曰∶按劉熙《釋名》雲∶雪，洗也。洗除瘴疠蟲蝗也。凡花五出，雪花六出，陰之成數也。冬至後第...,（宋《嘉 》）\n【釋名】時珍曰∶按劉熙《釋名》雲∶雪，洗也。洗除瘴疠蟲蝗也。凡花五出，雪花...,,（宋《嘉 》）\n【釋名】時珍曰∶按劉熙《釋名》雲∶雪，洗也。洗除瘴疠蟲蝗也。凡花五出，雪花...,解一切毒，治天行時氣溫疫，小兒熱痫狂啼，大人丹石發動，酒後暴熱，黃膽，仍小溫服之（藏器）。洗...,甘，冷，無毒。\n,
8,水部第五卷 水之一,燒酒,時珍曰∶程子雲∶雹者陰陽相搏之氣，蓋 氣也。或雲∶雹者，炮也，中物如炮也。曾子雲∶陽之專氣爲...,（音駁。《拾遺》）\n【釋名】時珍曰∶程子雲∶雹者陰陽相搏之氣，蓋 氣也。或雲∶雹者，炮也，...,,（音駁。《拾遺》）\n【釋名】時珍曰∶程子雲∶雹者陰陽相搏之氣，蓋 氣也。或雲∶雹者，炮也，...,,鹹，冷，有毒。時珍曰∶按《五雷經》雲∶人食雹，患疫疾大風颠邪之證。藏器曰∶醬味不正者，當時取...,
9,水部第五卷 水之一,燒酒,淩（去聲）。時珍曰∶冰者，太陰之精，水極似土，變柔爲剛，所謂物極反兼化 也。故字從水，從 。...,（《拾遺》）\n【釋名】淩（去聲）。時珍曰∶冰者，太陰之精，水極似土，變柔爲剛，所謂物極反兼...,,（《拾遺》）\n【釋名】淩（去聲）。時珍曰∶冰者，太陰之精，水極似土，變柔爲剛，所謂物極反兼...,去熱煩，熨人乳石發熱腫（藏器）。解煩渴，消暑毒（吳瑞）。傷寒陽毒，熱盛昏迷者，以冰一塊置于膻...,甘，冷，無毒。\n,新一。滅瘢痕∶以凍淩頻熨之，良。（《千金方》）


In [ ]:
new_df.to_csv('./data/本草綱目_Ver2.csv', encoding='utf-8-sig')

In [ ]:
df = new_df

In [ ]:
c_contents = df.loc[:,'附方']

In [ ]:
compounds = []
for c_content in c_contents:
    if pd.isnull(c_content):
        continue
    pattern = r'[。）》\n][\w ]*?( ?\w+[\w ，、]{1,10})∶'
    finds = re.findall(pattern, c_content, re.S)
    if finds:
        compounds.extend(finds)
#         print(finds)
print(len(set(compounds)))
# print(compounds)
[c for c in set(compounds) if len(c)>10 and all(x in c for x in ['、'])]

In [ ]:
# (?=(?=(?:' + compound_join + ')∶)|(?<=.$))
# r'''(?:(?<=(?=\w))|(?:（|《|》|）|，|。))\w*? ?(\w+[ ，\w]+)∶(.*?)
# (?=(?=(?:(?:(?<=(?=\w))|(?:（|《|》|）|，|。))\w*? ?(\w+[ ，\w]+)∶(.*?)))|(?<=.$))'''
compounds = []

for c_content in c_contents[9:20]:
    if pd.isnull(c_content):
        continue
    pattern = r'[。）》\n][\w ]*?( ?\w+[\w ，、]{1,10})∶(.*?)(?=(?:[。）》\n][\w ]*?(?: ?\w+[\w ，、]{1,10})∶)|(?<=.$))'
    
    finds = re.findall(pattern, c_content, re.M)
    if finds:
        compounds.extend(finds)
        print(finds, '\n')
        print(c_content, '\n')

print(len(set(compounds)))

In [ ]:
string = '''新三。目不得瞑∶乃陽氣盛不得入于陰，陰氣虛，故目不得瞑。治法飲以半夏湯，用流水千裏外者八升，揚之萬遍，取其清五升煮之，炊葦薪火，置秫米一升，半夏五合，徐炊令竭爲一升半，去滓飲汁一小杯，日三飲，以知爲度。詳半夏下。（《靈樞經》）汗後奔豚∶茯苓桂枝甘草大棗湯，治發汗後臍下悸，欲作奔豚者。茯苓一兩，炙甘草二錢半，桂枝三錢，大棗二枚。以甘瀾水二升，先煮茯苓，納諸藥煮服之，日三。（張仲景《金匱要略》）
服藥過劑煩悶∶東流水飲一、二升。（《肘後方》）'''

In [ ]:
# pattern = r'[。）》]((?: ?\w{0,10}[ ，、]?\n?){0,1}' \
#           r'(?: ?\w{0,2}[，、] \n){0,2}' \
#           r'(?:(?:\w+[，、]){1,3} ?\n?){0,3}\w{1,11})∶'

pattern = r'[。）》\n][\w ]*?( ?\w+[\w ，、]{1,10})∶(.*?)(?=(?:[。）》\n][\w ]*?(?: ?\w+[\w ，、]{1,10})∶)|(?<=.$))'

(re.findall(pattern, string, re.M))